In [3]:
import os 
import numpy as np
import pandas as pd
from typing  import List
from sklearn.datasets import load_iris

ROOT= "iris_course"
ARTIFACTS = os.path.join(ROOT,'artifacts')
os.makedirs(ARTIFACTS,exist_ok=True)

# axis=0 與 axis=1 差別
# print("axis=0：每欄平均", X.mean(axis=0))
# print("axis=1：每列平均", X.mean(axis=1))

# 這段的作用
# 這段是 把特徵名稱 (names)、平均值 (m)、標準差 (s) 串在一起逐一輸出。

# 📌 詳細拆解
# zip(names, m, s)
# names 是特徵名稱的字串清單，例如：
# ["sepal_length","sepal_width","petal_length","petal_width"]

# m 是各欄的平均值陣列，例如：
# [5.84, 3.05, 3.76, 1.20]

# s 是各欄的標準差陣列，例如：
# [0.82, 0.43, 1.76, 0.76]

# zip() 會把它們一一對應打包成 tuple：
# [
#   ("sepal_length", 5.84, 0.82),
#   ("sepal_width",  3.05, 0.43),
#   ...
# ]

# 格式	     說明
# {n:<14s}	左對齊字串，寬度 14
# {mi:8.4f}	浮點數，總寬度 8，顯示 4 位小數
# {sd:8.4f}	同上

def describe_stats(X:np.ndarray,names:List[str],title:str):
    m,s = X.mean(axis=0), X.std(axis=0)
    print(f"\n[{title}]")
    for n, mi, sd in zip(names, m,s):
        print(f"{n:<14s} mean={mi:8.4f} std={sd:8.4f}" )

print("***Step1 載入資料與探索")
iris =load_iris()
# print(iris)


# 載入資料集：
# x 是 (150,4) 的數值矩陣
# y 是 (150,) 的標籤（0,1,2）
# 這四個特徵分別是：花萼長寬、花瓣長寬
x,y = iris.data,iris.target

# sepal 萼片
# petal 花瓣
feature_names = ["sepal_length","sepal_width","petal_length","petal_width"]
target_names=iris.target_names.tolist()

# 這行會用 pandas 把 x 轉成一個 DataFrame（表格格式），欄名就是特徵名稱：
df=pd.DataFrame(x,columns=feature_names)
df["target"] = y
print("\n 前5筆資料");print(df.head())
print("\n 類別分布:")


# enumerate() 是 Python 內建函式
# 它的作用是：在迴圈中同時取得「索引」和「元素」

# 小總結
#表達式	                           意思	結果型態
# y	類別標籤陣列	           ndarray of int
# y == i	          元素逐一是否等於 i	ndarray of bool
# (y == i).sum()	      有幾個等於 i	int（計數結果）

# ✅ 所以雖然 y 是個整數陣列 (ndarray)，
# 用 == 去比對數值時，會自動對每個元素做比較，這就是 NumPy 的「向量化運算」。

# target_names：只會是 三個獨特的名稱 → ['setosa','versicolor','virginica']
for i ,name in enumerate(target_names):
    print(f"{i}={name:<10s} : {(y==i).sum()} 筆")
describe_stats(x,feature_names,"原始資料(未標準化)")

out_csv=os.path.join(ARTIFACTS,"iris_preview.csv")
# index=False 表示 不要輸出 DataFrame 的索引欄位（只保留資料本身）
df.head(20).to_csv(out_csv,index=False)

print(f"\n->已存取20筆預覽 :{out_csv}")
print("STEP1 完成")

***Step1 載入資料與探索

 前5筆資料
   sepal_length  sepal_width  petal_length  petal_width  target
0           5.1          3.5           1.4          0.2       0
1           4.9          3.0           1.4          0.2       0
2           4.7          3.2           1.3          0.2       0
3           4.6          3.1           1.5          0.2       0
4           5.0          3.6           1.4          0.2       0

 類別分布:
0=setosa     : 50 筆
1=versicolor : 50 筆
2=virginica  : 50 筆

[原始資料(未標準化)]
sepal_length   mean=  5.8433 std=  0.8253
sepal_width    mean=  3.0573 std=  0.4344
petal_length   mean=  3.7580 std=  1.7594
petal_width    mean=  1.1993 std=  0.7597

->已存取20筆預覽 :iris_course\artifacts\iris_preview.csv
STEP1 完成


In [4]:
from sklearn.model_selection import train_test_split
print("\n===STEP2 | 切分 Train/Val/Test===")



# x：所有的特徵資料（numpy.ndarray，shape = (150, 4)）
# y：所有的標籤（numpy.ndarray，shape = (150,)）
# train_test_split 是 scikit-learn 提供的
# train_test_split 函式，用來隨機把資料拆成兩組
# 參數解釋
# 參數	                            用途
# x, y	             輸入資料與標籤
# test_size=0.2	    指定 20% 當「測試集」，剩下 80% 是「訓練+驗證」
# random_state=42	設定隨機種子，讓結果可重現
# stratify(分層)=y	    分層抽樣：確保三個類別在兩組中比例一致

X_trainval,X_test,y_trainval,y_test =train_test_split(
    x,y,test_size=0.2,random_state=42,stratify=y
)
X_train,X_val,y_train,y_val =train_test_split(
    X_trainval,y_trainval,test_size=0.2,random_state=42,stratify=y_trainval
)

print(f"切分形狀: train={X_train.shape} val={X_val.shape} test={X_test.shape}")
print("STEP2 完成")


===STEP2 | 切分 Train/Val/Test===
切分形狀: train=(96, 4) val=(24, 4) test=(30, 4)
STEP2 完成


In [5]:
from sklearn.preprocessing import StandardScaler
import joblib

print('\====STEP 3 | 標準化(只用訓練集fit)並存檔)')

# StandardScaler 是一個「轉換器 (transformer)」物件
# 它會計算：
# 每個欄位的平均值 μ
# 每個欄位的標準差 σ
# 然後把資料套用公式：

# 𝑧 = (x - 𝜇) / 𝜎
# 讓 每個特徵（欄）都變成平均 0、標準差 1	​


# 第 1 行：先用訓練資料「學習平均與標準差」
# .fit(X_train) 會計算：
# 每一欄的平均值 mean_
# 每一欄的標準差 scale_
# 這一步「只用訓練集」是為了避免資料洩漏（不能偷看驗證或測試資料）
scaler = StandardScaler().fit(X_train)

# 第 2 行：把訓練資料做標準化
# 用剛剛算出的 mean_ 和 scale_ 把資料轉換成：
# (原值 - 平均) / 標準差
# 結果：每一欄的平均會變成 0、標準差變成 1
X_train_sc = scaler.transform(X_train)

# 第 3～4 行：用同一個 scaler 處理驗證與測試資料
# 這裡 不能再 fit 一次，要用 訓練集的平均與標準差 來轉換
# 這樣才確保模型在驗證/測試時使用完全相同的尺度
X_val_sc = scaler.transform(X_val)
X_test_sc = scaler.transform(X_test)

describe_stats(X_train, feature_names,"訓練集(標準化前)")
describe_stats(X_train_sc, feature_names,"訓練集(標準化後)")

npz_path= os.path.join(ARTIFACTS,"train_val_test_scaled.npz")
# .npz 就是 把很多 NumPy 陣列一起打包壓縮存檔
# → 讓你之後可以 一次存、一包讀，很方便。

# 這行會建立一個 train_val_test_scaled.npz 檔，裡面包含：

# 存進去的名稱	內容
# X_train_sc	標準化後的訓練特徵資料
# y_train	訓練標籤
# X_val_sc	標準化後的驗證特徵資料
# y_val	驗證標籤
# X_test_sc	標準化後的測試特徵資料
# y_test	測試標籤
# feature_names	特徵名稱清單（轉成陣列存）
# target_names	類別名稱清單（轉成陣列存）

np.savez(npz_path,
         X_train_sc=X_train_sc,y_train=y_train,
         X_val_sc=X_val_sc,y_val=y_val,
         X_test_sc=X_test_sc,y_test=y_test,
         feature_names=np.array(feature_names,dtype=object),
         target_names=np.array(target_names,dtype=object),
)

# 這兩行的目的

# 把你訓練好的 StandardScaler 物件
# 存成一個檔案（scaler.pkl），
# 以後要用時可以直接載回來，不用重新 .fit() 一次。

# 把物件存起來
# joblib.dump(scaler, scaler_path)
# 使用 joblib 的 dump 函式
# 把 scaler（也就是你用 X_train .fit() 過的 StandardScaler）存成 .pkl 檔案
# .pkl 是「pickle」格式，用來存整個 Python 物件

scaler_path =os.path.join(ARTIFACTS,"scaler.pkl")
joblib.dump(scaler,scaler_path)

print(f"->已存標準化資料:{npz_path}")
print(f"->已存標準化器:{scaler_path}")
print("STEP3 完成")

\====STEP 3 | 標準化(只用訓練集fit)並存檔)

[訓練集(標準化前)]
sepal_length   mean=  5.8333 std=  0.8516
sepal_width    mean=  3.0083 std=  0.4264
petal_length   mean=  3.7500 std=  1.7530
petal_width    mean=  1.1875 std=  0.7463

[訓練集(標準化後)]
sepal_length   mean=  0.0000 std=  1.0000
sepal_width    mean=  0.0000 std=  1.0000
petal_length   mean=  0.0000 std=  1.0000
petal_width    mean=  0.0000 std=  1.0000
->已存標準化資料:iris_course\artifacts\train_val_test_scaled.npz
->已存標準化器:iris_course\artifacts\scaler.pkl
STEP3 完成


In [6]:
import torch
from torch.utils.data import TensorDataset,DataLoader
# 整體目標
# 這段是 STEP4：把資料轉成 torch.Tensor，再包進 DataLoader，用來訓練模型
# 是 PyTorch 的標準資料處理流程。
# 簡單來說：
# 把資料(numpy.ndarray) → 轉成 Tensor → 用 DataLoader 分成小批次（batch） → 之後可以丟給模型訓練
print('\====STEP 4 | Tensor 與 DataLoader')


X_train_t = torch.tensor(X_train_sc,dtype=torch.float32)
y_train_t = torch.tensor(y_train,dtype=torch.long)
X_val_t = torch.tensor(X_val_sc,dtype=torch.float32)
y_val_t = torch.tensor(y_val,dtype=torch.long)


# TensorDataset(X, y)：把 X 和 y 包成一筆一筆的資料
# DataLoader(..., batch_size=16)：每次會吐出 16 筆資料（小批次）
# shuffle=True：訓練集會隨機打亂順序（避免模型記住順序）
# shuffle=False：驗證集保持原順序
# 為什麼驗證/測試集不要 shuffle
# 驗證時只是「評估」模型表現，不需要也不應打亂
# 保持固定順序 → 方便對照預測與真實標籤
# 每次評估結果一致，避免隨機性影響評估
train_loader =DataLoader(TensorDataset(X_train_t,y_train_t),batch_size=16,shuffle=True)
val_loader =DataLoader(TensorDataset(X_val_t,y_val_t),batch_size=16,shuffle=False)


# 目的

# 從 DataLoader（PyTorch）
# 取出「第一個 batch（小批次）」的資料，
# 用來 檢查資料長相是否正確。


# train_loader 是你剛建立的 DataLoader，裡面有所有訓練資料（已分好 batch）
# iter(train_loader) → 建立一個「迭代器」
# next(...) → 從迭代器中取出第一組 (特徵, 標籤)
# 結果會是：
# xb = 一個 batch 的特徵資料
# shape 通常是 (batch_size, 特徵數)
# 例如 (16, 4)
# yb = 一個 batch 的標籤資料
# shape 是 (batch_size,)
# 例如 (16,)

# 重點觀念
# shuffle=True 的 打亂時機不是在你建立 DataLoader 的當下，
# 而是在你第一次呼叫 iter(train_loader)（開始一個 epoch）時才打亂資料順序。

xb,yb = next(iter(train_loader))
print(f"第一個 batch:xb.shape={xb.shape}, yb.shape={yb.shape}")

# 取出 batch 裡第一筆資料的標籤
print(f"   xb[0](標準化後)={xb[0].tolist()}")
print(f"   yb[0](類別)={yb[0].item()}")


# 功能（一句話）
# 把你剛從 DataLoader 取出的那一個 batch（小批次）
# 轉成表格（pandas.DataFrame），加上標籤欄位，
# 再存成 CSV 檔，方便你用 Excel 或其他工具檢查。
batch_preview=os.path.join(ARTIFACTS,"batch_preview.csv")
pd.DataFrame(xb.numpy(),columns=feature_names).assign(label=yb.numpy()).to_csv(batch_preview,index=False)
print(f"->已存batch 預覽{batch_preview}")
print("STEP4 完成")

\====STEP 4 | Tensor 與 DataLoader
第一個 batch:xb.shape=torch.Size([16, 4]), yb.shape=torch.Size([16])
   xb[0](標準化後)=[1.017750859260559, -1.1921887397766113, 1.1694414615631104, 0.8207424879074097]
   yb[0](類別)=2
->已存batch 預覽iris_course\artifacts\batch_preview.csv
STEP4 完成


In [7]:
from torch import nn 
MODELS = os.path.join(ROOT,"models")
os.makedirs(MODELS,exist_ok=True)

print('\====STEP 5 | 定義模型與參數量')

class IrisMLP(nn.Module):
    def __init__(self, in_dim=4, hidden1=64,hidden2=32,out_dim=3,dropout=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim,hidden1),nn.ReLU(),nn.Dropout(dropout),
            nn.Linear(hidden1,hidden2),nn.ReLU(),nn.Dropout(dropout),
            nn.Linear(hidden2,out_dim)
        )


    #     forward：定義前向傳遞
    # def forward(self, x):
    #     return self.net(x)


    # 告訴 PyTorch 資料要怎麼經過網路

    # 只要呼叫 model(x)，就會自動跑這段
    def forward(self,x): return self.net(x)




# 計算 PyTorch 模型中「可訓練參數」的總數
# 也就是：這個模型裡 所有需要更新的權重參數 一共有幾個數值（weights / biases）。

# 你問的 -> int 是 Type Hint（型別註解） 的一種，
# 不是程式功能的一部分，只是「告訴人或工具：這個函式會回傳什麼型別」。
def count_trainable_params(model: nn.Module) -> int:

    #     p.numel()
    # 回傳這個 Tensor 裡「有幾個元素」
    # 例如：
    # p.shape = (64, 4) → p.numel() = 256
    # p.shape = (64,)   → p.numel() = 64

    #     (p.numel() for p in ...) 是 生成器，不是陣列，
    # 它會「一個一個產生數字」，讓 sum() 去加總，
    # 而不會先把所有數字存在記憶體裡。

    # requires_grad 是什麼
    # 它的意思是：
    # 這個張量是否要在反向傳播（backpropagation）時計算梯度
    # 有些參數可能：
    # 是凍結的（不想訓練）
    # 是固定的 embedding 或預訓練權重
    return sum(p.numel() for p in model.parameters() if p.requires_grad)



# 檢查你的電腦有沒有 NVIDIA GPU（CUDA）可以用，
# 然後設定一個 torch.device 物件，
# 讓你之後可以把模型或資料移到 GPU 或 CPU 執行。
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 建立模型物件
# 把模型搬到指定的裝置（CPU / GPU）上
model = IrisMLP().to(device)
print(model)
print(f"可訓練參數量:{count_trainable_params(model):,}")

arch_txt =os.path.join(MODELS,"model_arch.txt")

# 功能總覽

# 把 模型的結構 和 可訓練參數總數
# 寫進一個文字檔（arch_txt）

# 用 Python 內建的檔案操作
# "w" → 以「寫入模式」開啟檔案
# encoding="utf-8" → 用 UTF-8 編碼（可以正常寫中文）
# as f → 建立檔案物件 f
# with 會在寫完後自動關閉檔案

with open(arch_txt,"w",encoding="utf-8") as f:
    f.write(str(model) + "\n")
    f.write(f"trainable_params={count_trainable_params(model)}\n")
print(f"-> 已存結構描述:{arch_txt}")
print("STEP5 完成") 

\====STEP 5 | 定義模型與參數量
IrisMLP(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=32, out_features=3, bias=True)
  )
)
可訓練參數量:2,499
-> 已存結構描述:iris_course\models\model_arch.txt
STEP5 完成


c:\Users\User\anaconda3\envs\pyTorch\lib\site-packages\torch\cuda\__init__.py:218: UserWarning: 
NVIDIA GeForce RTX 5080 with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5080 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(
